# ALIGN Tutorial Notebook: CHILDES

This notebook provides an introduction to **ALIGN**, 
a tool for quantifying multi-level linguistic similarity 
between speakers, using parent-child transcript data from 
the Kuczaj Corpus 
(https://childes.talkbank.org/access/Eng-NA/Kuczaj.html).

This method was introduced in "ALIGN: Analyzing Linguistic Interactions with Generalizable techNiques" (Duran, Paxton, & Fusaroli, *submitted*).

## Tutorial Overview

While many studies of interpersonal linguistic alignment
compare alignment between different dyads across different
conditions (i.e., typically a within- or between-dyads
design in which each dyad contributes only one or two 
conversations), there may also be interest in understanding
longer-scale temporal dynamics *within* a given dyad.
This tutorial provides an example of how ALIGN may be used
to just that end: analyzing how a single dyad's multilevel
alignment changes across different conversations held
at different points over a longer range of time.

To do so, this tutorial walks users through an analysis of
conversations from a single English corpus from the CHILDES 
database  (MacWhinney, 2000)---specifically, Kuczaj’s Abe 
corpus (Kuczaj, 1976), used under a Creative Commons 
Attribution-ShareAlike 3.0 Unported License (see GitHub
repository or `data/CHILDES` directory for license). 
We analyze the last 20 conversations in the corpus in order
to explore how ALIGN can be used to track multi-level
linguistic alignment between a parent and child over time,
which may be of interest to developmental language
researchers. Specifically, we explore how alignment between a parent
and a child changes over a brief span of developmental
trajectory.

Data for this tutorial are shipped with the `align`
package on PyPI (https://pypi.python.org/pypi/align) and GitHub
(https://github.com/nickduran/align-linguistic-alignment/).

***

## Table of Contents

* [Getting Started](#Getting-Started)
    * [Prerequisites](#Prerequisites)
    * [Preparing input data](#Preparing-input-data)
    * [Filename conventions](#Filename-conventions)
    * [Highest-level functions](#Highest-level-functions)
* [Setup](#Setup)
    * [Import libraries](#Import-libraries)
    * [Specify ALIGN path settings](#Specify-ALIGN-path-settings)
* [Phase 1: Prepare transcripts](#Phase-1:-Prepare-transcripts)
    * [Preparation settings](#Preparation-settings)
    * [Run preparation phase](#Run-preparation-phase)
* [Phase 2: Calculate alignment](#Phase-2:-Calculate-alignment)
    * [For real data: Alignment calculation settings](#For-real-data:-Alignment-calculation-settings)
    * [For real data: Run alignment calculation](#For-real-data:-Run-alignment-calculation)
    * [For surrogate data: Alignment calculation settings](#For-surrogate-data:-Alignment-calculation-settings)
    * [For surrogate data: Run alignment calculation](#For-surrogate-data:-Run-alignment-calculation)
* [ALIGN output overview](#ALIGN-output-overview)
    * [Speed calculations](#Speed-calculations)
    * [Printouts!](#Printouts!)

***

# Getting Started

## Preparing input data

* Each input text file needs to contain a single conversation organized in an `N x 2` matrix
    * Text file must be tab-delimited.
* Each row must correspond to a single conversational turn from a speaker.
    * Rows must be temporally ordered based on their occurrence in the conversation.
    * Rows must alternate between speakers.
* Speaker identifier and content for each turn are divided across two columns.
    * Column 1 must have the header `participant`.
        * Each cell specifies the speaker.
        * Each speaker must have a unique label (e.g., `P1` and `P2`, `0` and `1`).
    * Column 2 must have the header `content`.
        * Each cell corresponds to the transcribed utterance from the speaker.
        * Each cell must end with a newline character: `\n`
* See `examples` directory in Github repository for an example

## Filename conventions

* Each conversation text file must be regularly formatted, including a prefix for dyad and a prefix for conversation prior to the identifier for each that are separated by a unique character. By default, ALIGN looks for patterns that follow this convention: `dyad1-condA.txt`
    * However, users may choose to include any label for dyad or condition so long as the two labels are distinct from one another and are not subsets of any possible dyad or condition labels. Users may also use any character as a separator so long as it does not occur anywhere else in the filename.
    * The chosen file format **must** be used when saving **all** files for this analysis.

## Highest-level functions

Given appropriately prepared transcript files, ALIGN can be run in 3 high-level functions:

**`prepare_transcripts`**: Pre-process each standardized 
conversation, checking it conforms to the requirements. 
Each utterance is tokenized and lemmatized and has 
POS tags added.

**`calculate_alignment`**: Generates turn-level and 
conversation-level alignment scores (lexical, 
conceptual, and syntactic) across a range of 
*n*-gram sequences.

**`calculate_baseline_alignment`**: Generate a surrogate corpus
and run alignment analysis (using identical specifications 
from `calculate_alignment`) on it to produce a baseline.

***

# Setup

## Import libraries

Import packages we'll need to run ALIGN.

In [63]:
import align, os
import pandas as pd

Import `time` so that we can get a sense of how
long the ALIGN pipeline takes.

In [64]:
import time

Import `warnings` to flag us if required files aren't provided.

In [65]:
import warnings

Load in the `rpy2.ipython` Jupyter notebook extension so
that we can run R analysis in notebook with a Python kernel.

In [66]:
# %load_ext rpy2.ipython

**ALEX** I don't think this is necessary as the R analysis code is going to be hosted elsewhere. I don't want to deal setting it up to run here in the notebook. Maybe down the road... 

## Install additional NTLK packages

Download some addition `nltk` packages for `align` to work.

In [67]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/nduran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nduran/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/nduran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**ALEX** Was throwing import error because of a typo in spelling in "nltk" - now fixed

## Specify ALIGN path settings

ALIGN will need to know where the raw transcripts are stored, where to store the processed data, and where to read in any additional files needed for optional ALIGN parameters.

### Required directories

For the sake of this tutorial, specify a base path that will serve as our jumping-off point for our saved data. All of the CHILDES data and shipped data will be called from the package directory.

**`BASE_PATH`**: Containing directory for this tutorial.

In [68]:
BASE_PATH = os.getcwd()

**`CHILDES_EXAMPLE`**: Subdirectories for output and other
files for this tutorial. (We'll create a default directory
if one doesn't already exist.)

In [69]:
CHILDES_EXAMPLE = os.path.join(BASE_PATH,
                              'CHILDES/')

In [70]:
if not os.path.exists(CHILDES_EXAMPLE):
    os.makedirs(CHILDES_EXAMPLE)

**`TRANSCRIPTS`**: Path to raw transcript files. Automatically provided by `align`.

In [72]:
TRANSCRIPTS = align.datasets.CHILDES_directory

**`PREPPED_TRANSCRIPTS`**: Set variable for folder name 
(as string) for relative location of folder into which 
prepared transcript files will be saved. (We'll create
a default directory if one doesn't already exist.)

In [46]:
PREPPED_TRANSCRIPTS = os.path.join(CHILDES_EXAMPLE,
                                   'childes-prepped/')

In [47]:
if not os.path.exists(PREPPED_TRANSCRIPTS):
    os.makedirs(PREPPED_TRANSCRIPTS)

**`ANALYSIS_READY`**: Set variable for folder name 
(as string) for relative location of folder into 
which analysis-ready dataframe files will be saved.
(We'll create a default directory if one doesn't
already exist.)

In [48]:
ANALYSIS_READY = os.path.join(CHILDES_EXAMPLE,
                              'childes-analysis/')

In [49]:
if not os.path.exists(ANALYSIS_READY):
    os.makedirs(ANALYSIS_READY)

**`SURROGATE_TRANSCRIPTS`**: Set variable for folder name 
(as string) for relative location of folder into which all
prepared surrogate transcript files will be saved. (We'll
create a default directory if one doesn't already exist.)

In [50]:
SURROGATE_TRANSCRIPTS = os.path.join(CHILDES_EXAMPLE,
                                     'childes-surrogate/')

In [51]:
if not os.path.exists(SURROGATE_TRANSCRIPTS):
    os.makedirs(SURROGATE_TRANSCRIPTS)

### Paths for optional parameters

**`OPTIONAL_PATHS`**: If using Stanford POS tagger or
pretrained vectors, the path to these files. If these
files are provided in other locations, be sure to
change the file paths for them. (We'll create a default
directory if one doesn't already exist.)

In [52]:
OPTIONAL_PATHS = os.path.join(CHILDES_EXAMPLE,
                             'optional_directories/')

In [53]:
if not os.path.exists(OPTIONAL_PATHS):
    os.makedirs(OPTIONAL_PATHS)

#### Stanford POS Tagger

The Stanford POS tagger **will not be used** by 
default in this example. However, you may use them
by uncommenting and providing the requested file 
paths in the cells in this section and then changing 
the relevant parameters in the ALIGN calls below.

If desired, we could use the Standford part-of-speech 
tagger along with the Penn part-of-speech tagger
(which is always used in ALIGN). To do so, the files
will need to be downloaded separately: 
https://nlp.stanford.edu/software/tagger.shtml#Download

**`STANFORD_POS_PATH`**: If using Stanford POS tagger
with the Penn POS tagger, path to Stanford directory.

In [54]:
# STANFORD_POS_PATH = os.path.join(OPTIONAL_PATHS,
#                                  'stanford-postagger-full-2017-06-09/')

In [55]:
# if not STANFORD_POS_PATH:
#     warnings.warn('Stanford POS directory not found at the specified '
#                   'location. Please update the file path or comment '
#                   'out the `STANFORD_POS_PATH` information.')

**`STANFORD_LANGUAGE`**: If using Stanford tagger,
set language model to be used for POS tagging.

In [56]:
# STANFORD_LANGUAGE = os.path.join(OPTIONAL_PATHS,
#                                  'english-left3words-distsim.tagger')

In [57]:
# if not STANFORD_LANGUAGE:
#     warnings.warn('Stanford tagger language not found at the specified '
#                   'location. Please update the file path or comment '
#                   'out the `STANFORD_POS_PATH` information.')

#### Google News pretrained vectors

The Google News pretrained vectors **will be used**
by default in this example. The file is available for
download here: https://code.google.com/archive/p/word2vec/

If desired, researchers may choose to read in pretrained
`word2vec` vectors rather than creating a semantic space
from the corpus provided. This may be especially useful 
for small corpora (i.e., fewer than 30k unique words),
although the choice of semantic space corpus should be
made with careful consideration about the nature of the
linguistic context (for further discussion, see Duran, 
Paxton, & Fusaroli, *submitted*).

**`PRETRAINED_INPUT_FILE`**: If using pretrained vectors, path
to pretrained vector files. You may choose to download the file
directly to this path or change the path to a different one.

In [58]:
PRETRAINED_INPUT_FILE = os.path.join(OPTIONAL_PATHS,
                            'GoogleNews-vectors-negative300.bin')

In [59]:
if not PRETRAINED_INPUT_FILE:
    warnings.warn('Google News vector not found at the specified '
                  'location. Please update the file path or comment '
                  'out the `PRETRAINED_INPUT_FILE` information.')

***

# Phase 1: Prepare transcripts

In Phase 1, we take our raw transcripts and get them ready
for later ALIGN analysis.

## Preparation settings

There are a number of parameters that we can set for the
`prepare_transcripts()` function:

In [60]:
print align.prepare_transcripts.__doc__


    Prepare transcripts for similarity analysis.

    Given individual .txt files of conversations,
    return a completely prepared dataframe of transcribed
    conversations for later ALIGN analysis, including: text
    cleaning, merging adjacent turns, spell-checking,
    tokenization, lemmatization, and part-of-speech tagging.
    The output serve as the input for later ALIGN
    analysis.

    Parameters
    ----------

    input_files : str (directory name) or list of str (file names)
        Raw files to be cleaned. Behavior governed by `input_as_directory`
        parameter as well.

    output_file_directory : str
        Name of directory where output for individual conversations will be
        saved.

    training_dictionary : str, optional (default: None)
        Specify whether to train the spell-checking dictionary using a
        provided file name (str) or the default Project
        Gutenberg corpus [http://www.gutenberg.org] (None).

    minwords : int, optional (2)

For the sake of this demonstration, we'll keep everything as
defaults. Among other parameters, this means that:
* any turns fewer than 2 words will be removed from the corpus
 (`minwords=2`),
* we'll be using regex to strip out any filler words
 (e.g., "uh," "um," "huh"; `use_filler_list=None`),
* we'll be using the Project Gutenberg corpus to create our 
 spell-checker algorithm (`training_dictionary=None`),
* we'll rely only on the Penn POS tagger 
 (`add_stanford_tags=False`), and
* our data will be saved both as individual conversation files
 and as a master dataframe of all conversation outputs
 (`save_concatenated_dataframe=True`).

## Run preparation phase

First, we prepare our transcripts by reading in individual `.txt`
files for each conversation, clean up undesired text and turns,
spell-check, tokenize, lemmatize, and add POS tags.

In [61]:
start_phase1 = time.time()

In [62]:
model_store = align.prepare_transcripts(
                        input_files=TRANSCRIPTS,
                        output_file_directory=PREPPED_TRANSCRIPTS,
                        minwords=2,
                        use_filler_list=None,
                        training_dictionary=None,
                        add_stanford_tags=False,
                        save_concatenated_dataframe=True)

IOError: [Errno 2] No such file or directory: '/Users/nduran/anaconda/lib/python2.7/site-packages/align/data/gutenberg.txt'

**ALEX** Do you know what is generating this error? It looks like the gutenberg.txt file is not saved with the align package on the pip install. Thanks.  

In [ ]:
end_phase1 = time.time()

***

# Phase 2: Calculate alignment

## For real data: Alignment calculation settings

There are a number of parameters that we can set for the
`calculate_alignment()` function:

In [ ]:
print align.calculate_alignment.__doc__

For the sake of this tutorial, we'll keep everything as
defaults. Among other parameters, this means that we'll:
* use only unigrams and bigrams for our *n*-grams
 (`maxngram=2`),
* use pretrained vectors instead of creating our own
 semantic space, since our tutorial corpus is quite
 small (`use_pretrained_vectors=True` and
 `pretrained_file_directory=PRETRAINED_INPUT_FILE`),
* ignore exact lexical duplicates when calculating
 syntactic alignment,
* we'll rely only on the Penn POS tagger 
 (`add_stanford_tags=False`), and
* implement high- and low-frequency cutoffs to clean
 our transcript data (`high_sd_cutoff=3` and 
 `low_n_cutoff=1`).

Whenever we calculate a baseline level of alignment,
we need to include the same parameter values for any
parameters that are present in both `calculate_alignment()`
(this step) and `calculate_baseline_alignment()`
(next step). As a result, we'll specify these here:

In [ ]:
# set standards to be used for real and surrogate
INPUT_FILES = PREPPED_TRANSCRIPTS
MAXNGRAM = 2
USE_PRETRAINED_VECTORS = True
SEMANTIC_MODEL_INPUT_FILE = os.path.join(CHILDES_EXAMPLE,
                                         'align_concatenated_dataframe.txt')
PRETRAINED_FILE_DRIRECTORY = PRETRAINED_INPUT_FILE
ADD_STANFORD_TAGS = False
IGNORE_DUPLICATES = True
HIGH_SD_CUTOFF = 3
LOW_N_CUTOFF = 1

## For real data: Run alignment calculation

In [ ]:
start_phase2real = time.time()

In [ ]:
[turn_real,convo_real] = align.calculate_alignment(
                            input_files=INPUT_FILES,
                            maxngram=MAXNGRAM,   
                            use_pretrained_vectors=USE_PRETRAINED_VECTORS,
                            pretrained_input_file=PRETRAINED_INPUT_FILE,
                            semantic_model_input_file=SEMANTIC_MODEL_INPUT_FILE,
                            output_file_directory=ANALYSIS_READY,
                            add_stanford_tags=ADD_STANFORD_TAGS,
                            ignore_duplicates=IGNORE_DUPLICATES,
                            high_sd_cutoff=HIGH_SD_CUTOFF,
                            low_n_cutoff=LOW_N_CUTOFF)

In [ ]:
end_phase2real = time.time()

## For surrogate data: Alignment calculation settings

For the surrogate or baseline data, we have many of the same
parameters for `calculate_baseline_alignment()` as we do for
`calculate_alignment()`:

In [ ]:
print align.calculate_baseline_alignment.__doc__

As mentioned above, when calculating the baseline, it is **vital** 
to include the *same* parameter values for any parameters that 
are included  in both `calculate_alignment()` and 
`calculate_baseline_alignment()`. As a result, we re-use those
values here.

We demonstrate other possible uses for labels by setting 
`dyad_label = time`, allowing us to compare alignment over 
time across the same speakers. We also demonstrate how to 
generate a subset of surrogate pairings rather than all 
possible pairings.

In addition to the parameters that we're re-using from
the `calculate_alignment()` values (see above), we'll 
keep most parameters at their defaults by:
* preserving the turn order when creating surrogate
 pairs (`keep_original_turn_order=True`),
* specifying condition with `cond` prefix 
 (`condition_label='cond'`), and
* using a hyphen to separate the condition and
 dyad identifiers (`id_separator='\-'`).
 
However, we will also change some of these defaults,
including:
* generating only a subset of surrogate data equal
 to the size of the real data (`all_surrogates=False`)
 and
* specifying that we'll be shuffling the baseline data
 by time instead of by dyad (`dyad_label='time'`).

## For surrogate data: Run alignment calculation

In [ ]:
start_phase2surrogate = time.time()

In [ ]:
[turn_surrogate,convo_surrogate] = align.calculate_baseline_alignment(
                                    input_files=INPUT_FILES, 
                                    maxngram=MAXNGRAM,
                                    use_pretrained_vectors=USE_PRETRAINED_VECTORS,
                                    pretrained_input_file=PRETRAINED_INPUT_FILE,
                                    semantic_model_input_file=SEMANTIC_MODEL_INPUT_FILE,
                                    output_file_directory=ANALYSIS_READY,
                                    add_stanford_tags=ADD_STANFORD_TAGS,
                                    ignore_duplicates=IGNORE_DUPLICATES,
                                    high_sd_cutoff=HIGH_SD_CUTOFF,
                                    low_n_cutoff=LOW_N_CUTOFF,
                                    surrogate_file_directory=SURROGATE_TRANSCRIPTS,
                                    all_surrogates=False,
                                    keep_original_turn_order=True,
                                    id_separator='\-',
                                    dyad_label='time',
                                    condition_label='cond')

In [ ]:
end_phase2surrogate = time.time()

***

# ALIGN output overview

## Speed calculations

As promised, let's take a look at how long it takes to run each section. Time is given in seconds.

**Phase 1:**

In [ ]:
end_phase1 - start_phase1

**Phase 2, real data:**

In [ ]:
end_phase2real - start_phase2real

**Phase 2, surrogate data:**

In [ ]:
end_phase2surrogate - start_phase2surrogate

**All phases:**

In [ ]:
end_phase2surrogate - start_phase1

## Printouts!

And that's it! Before we go, let's take a look at the output from the real data analyzed at the turn level for each conversation (`turn_real`) and at the conversation level for each dyad (`convo_real`). We'll then look at our surrogate data, analyzed both at the turn level (`turn_surrogate`) and at the conversation level (`convo_surrogate`). In our next step, we would then take these data and plug them into our statistical model of choice, but we'll stop here for the sake of our tutorial.

In [ ]:
turn_real.head(10)

In [ ]:
convo_real.head(10)

In [ ]:
turn_surrogate.head(10)

In [ ]:
convo_surrogate.head(10)